In [20]:
## Standard Stuff
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt
import dask.array as da
import numcodecs

## HEALPix Specific
import healpix as hp
import easygems.healpix as egh
import easygems.remap as egr

import intake     # For catalogs
import zarr

# Ilan
from icecream import ic
import nc_time_axis

def worldmap(var, title='', cbar_title='', **kwargs):
    #projection = ccrs.Robinson(central_longitude=-135.5808361)
    projection = ccrs.Robinson(central_longitude=0)
    fig, ax = plt.subplots(
        figsize=(8, 4), subplot_kw={"projection": projection}, constrained_layout=True
    )
    ax.set_global()
    ax.set_title(title)

    hpshow = egh.healpix_show(var, ax=ax, **kwargs)
    cbar = plt.colorbar(hpshow, ax=ax, orientation='vertical', 
                    pad=0.05, shrink=0.8, label=cbar_title)
    ax.add_feature(cf.COASTLINE, linewidth=0.8)
    ax.add_feature(cf.BORDERS, linewidth=0.4)
    
def usmap(var, title='', cbar_title='', **kwargs):
    #projection = ccrs.Robinson(central_longitude=-135.5808361)
    #projection = ccrs.Robinson(central_longitude=-90)
    projection = ccrs.PlateCarree()
    fig, ax = plt.subplots(
        figsize=(6, 3), subplot_kw={"projection": projection}, constrained_layout=True
    )
    ax.set_extent([-130, -60, 20, 45])
    
    hpshow = egh.healpix_show(var, ax=ax, **kwargs)
    cbar = plt.colorbar(hpshow, ax=ax, orientation='vertical', 
                    pad=0.05, shrink=0.7, label=cbar_title)
    ax.set_title(title)
    ax.add_feature(cf.COASTLINE, linewidth=0.8)
    #ax.coastlines(linewidth=0.8)
    ax.add_feature(cf.BORDERS, linewidth=0.4)
    ax.add_feature(cf.STATES, linewidth=0.4)

In [2]:
# set up dask
from dask.distributed import Client, LocalCluster
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()
# cluster = LocalCluster()
# client = Client(cluster)
# client

In [3]:
# for name in ['XSHIELD', 'ICON']:
#     llj = xr.open_dataset(f'/scratch/cimes/iv4111/hk25-data/llj_{name}.h5', chunks='auto')
#     freq = llj.mask.sum('time')/len(llj.time)*100
#     freq = freq.where(freq>0, np.nan)
#     title = f'{name} (2019-10-20 to 2021-1-7)'
#     usmap(freq, title=f'LLJ Occurrence {title}', cbar_title='Frequency [%]', vmin=0, vmax=40)
#     # Speed
#     usmap(llj.speed.mean('time'), title=f'LLJ Speed {title}', cbar_title=r'Mean Speed of LLJ [m s$^{-1}$]', vmin=0, vmax=25, cmap='plasma')
#     # Jet height
#     usmap(llj.height.mean('time'), title=f'LLJ Height {title}', cbar_title=r'Mean Height of LLJ [m]', vmin=0, vmax=1000, cmap='cividis')

In [25]:
xsh = xr.open_dataset(f'/scratch/cimes/iv4111/hk25-data/llj_XSHIELD.h5', chunks='auto')
xsh_freq = xsh.mask.sum('time')/len(xsh.time)*100>5
# xsh_freq = xr.where(xsh_freq>5, xsh_freq, np.nan)
# xsh = xr.where(xsh_freq>5, xsh,  np.nan)
print('Number of X-Shield Times: ', len(xsh.time))
for name in ['SCREAM', 'IFS', 'ICON', ]:
    ds = xr.open_dataset(f'/scratch/cimes/iv4111/hk25-data/llj_{name}.h5', chunks='auto')
    print(f'Number of {name} Times: ', len(ds.time))
    ds_freq = ds.mask.sum('time')/len(ds.time)*100
    # ds_freq = xr.where(ds_freq>5, ds_freq, np.nan)
    # ds = xr.where(ds_freq>5, ds, np.nan)
    # Need to ensure attributes
    # ds_freq['crs'].attrs = xsh['crs'].attrs.copy()
    # ds['crs'].attrs = xsh['crs'].attrs.copy()
    # Frequency
    freq_diff = ds.speed.mean('time') - xsh.speed.mean('time')
    freq_diff['crs'].attrs = xsh['crs'].attrs.copy()
    usmap(freq_diff, title=f'{name} - X-Shield LLJ Occurence', cbar_title=r'$\Delta$ [%]', vmin=-7.5, vmax=7.5, cmap='RdBu_r')
    plt.savefig(f'figures/deltaXSH_{name}_occurence.jpg', dpi=300)
    plt.close()
    # Speed
    speed_diff = ds.speed.mean('time') - xsh.speed.mean('time')
    usmap(speed_diff, title=f'{name} - X-Shield LLJ Speed', cbar_title=r'$\Delta$ [m s$^{-1}$]', vmin=-5, vmax=5, cmap='RdBu_r')
    plt.savefig(f'figures/deltaXSH_{name}_speed.jpg', dpi=300)
    plt.close()
    # Height
    height_diff = ds.height.mean('time') - xsh.height.mean('time')
    usmap(height_diff, title=f'{name} - X-Shield LLJ Height', cbar_title=r'$\Delta$ [m]', vmin=-400, vmax=400, cmap='RdBu_r')
    plt.savefig(f'figures/deltaXSH_{name}_height.jpg', dpi=300)
    plt.close()

Number of X-Shield Times:  3560
Number of SCREAM Times:  3168
[########################################] | 100% Completed | 103.02 ms
[########################################] | 100% Completed | 102.42 ms
[########################################] | 100% Completed | 101.65 ms
[########################################] | 100% Completed | 103.17 ms
[########################################] | 100% Completed | 837.63 ms
[########################################] | 100% Completed | 102.46 ms
[########################################] | 100% Completed | 103.57 ms
[########################################] | 100% Completed | 102.19 ms
[########################################] | 100% Completed | 103.60 ms
[########################################] | 100% Completed | 538.37 ms
[########################################] | 100% Completed | 102.43 ms
[########################################] | 100% Completed | 103.76 ms
[########################################] | 100% Completed | 102.12 ms
[#